In [5]:

# what if you do lda on the descriptions and use that to cluster the topics? once youve done that you can use the clusters match with embeddings of the whole description to find out which topics are most similar to the whole thing. you can then use these keywords to find the most similar industries.

InvalidRequestError: The model: `gpt-4` does not exist

In [20]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from tqdm import tqdm

In [72]:
# Load your dataframe
df = pd.read_csv(r'C:\Users\imran\DataspellProjects\WalidCase\data\processed/spacy_engineered/1k_nouns_adjectives.csv')
df.dropna(inplace=True)

# remove the rows where the description is less than 10 words
df = df[df['cb_description'].apply(lambda x: len(x.split()) > 30)]
df.dropna(inplace=True)

In [73]:
df = df.dropna()
# remove the rows where the description is too short or too long
#df = df[df['cb_description'].apply(lambda x: len(x.split()) > 30 and len(x.split()) < 100)]

# remove the column called id
df = df.drop(columns=['id'])
df = df.head(30).dropna()
df.reset_index(inplace=True, drop=True)



In [75]:
#df = pd.read_csv(r'C:\Users\imran\DataspellProjects\WalidCase\data\processed\GPT4_generated_keywords.csv')

df.shape

(30, 2)

In [65]:
# Step 1: Use a sentence transformer to generate word embeddings for each description
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
# Step 2: Use the embeddings to cluster words and produce a list of keywords for each cluster
nlp = spacy.load("en_core_web_sm")
df = df.head(30)

# Try with whole

In [79]:
#ignore warnings
import warnings
warnings.filterwarnings("ignore")

highest_scored_clusters = []
top_matched_cluster_keywords = []


for row, description in tqdm(df['cb_description'].iteritems()):
    word_list = []
    doc = nlp(description)
    words = [token.text for token in doc if token.is_alpha and not token.is_stop]
    word_list.extend(words)
    unique_words = list(set(word_list))
    word_embeddings = model.encode(unique_words)

    # Cluster the unique words using KMeans
    n_clusters = 3
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans_output = kmeans.fit_predict(word_embeddings) # produces a list of cluster labels for each word

    # Group the unique words by their assigned clusters
    word_clusters = {i: [] for i in range(n_clusters)}
    for i, label in enumerate(kmeans_output):
        word_clusters[label].append(unique_words[i])


    # Step 4: Use a sentence transformer to make embeddings for each cluster
    cluster_embeddings = []
    for i in range(n_clusters):
        cluster_text = ', '.join(word_clusters[i])
        cluster_embeddings.append(model.encode(cluster_text))

    description_embedding = model.encode([description])
    similarity_matrix = cosine_similarity(description_embedding, cluster_embeddings)
    highest_scored_cluster = np.argmax(similarity_matrix, axis=1)[0]
    highest_scored_clusters.append(highest_scored_cluster)
    top_matched_cluster_keywords.append(', '.join(word_clusters[highest_scored_cluster]))

df['highest_scored_cluster'] = highest_scored_clusters
df['top_matched_cluster_keywords'] = top_matched_cluster_keywords

30it [00:07,  4.16it/s]


# Try 1 - next sentence

In [80]:
df = df[['name', 'top_matched_cluster_keywords']]
df.rename(columns={'top_matched_cluster_keywords': 'cb_description'}, inplace=True)
df['id'] = df.index

               name                                     cb_description  id
0      InterResolve           insurance, insurer, agency, compensation   0
1         GladCloud  brand, asset, customisation, product, learn, m...   1
2          13th-Lab         camera, vision, sensor, vehicle, potential   2
3      Hilson-Moran                           leisure, facility, hotel   3
4  1928-Diagnostics             infection, diagnostic, trace, outbreak   4


In [81]:
df.to_csv(r'C:\Users\imran\DataspellProjects\WalidCase\data\processed\clustered/3_clusters_nouns_adjectives_only.csv', index=False)

# try 2 - using top words from keywords via tfidf

In [60]:
# Step 1: Use a sentence transformer to generate word embeddings for each description
model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

nlp = spacy.load("en_core_web_sm")

highest_scored_clusters = []
top_matched_cluster_keywords = []

# Function to extract top n words with highest tf-idf scores
def extract_top_n_words(corpus, n=10):
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(corpus)
    feature_names = np.array(vectorizer.get_feature_names_out())
    top_n_indices = np.argsort(tfidf_matrix.toarray())[:, -n:]
    top_n_words = feature_names[top_n_indices]
    return [', '.join(words) for words in top_n_words]

top_words_list = extract_top_n_words(df['cb_description'])

for row, (description, top_words) in enumerate(zip(df['cb_description'], top_words_list)):
    word_list = []
    doc = nlp(description)
    words = [token.text for token in doc if token.is_alpha and not token.is_stop]
    word_list.extend(words)
    unique_words = list(set(word_list))
    word_embeddings = model.encode(unique_words)

    # Cluster the unique words using KMeans
    n_clusters = 3
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans_output = kmeans.fit_predict(word_embeddings)

    # Group the unique words by their assigned clusters
    word_clusters = {i: [] for i in range(n_clusters)}
    for i, label in enumerate(kmeans_output):
        word_clusters[label].append(unique_words[i])

    # Step 4: Use a sentence transformer to make embeddings for each cluster
    cluster_embeddings = []
    for i in range(n_clusters):
        cluster_text = ', '.join(word_clusters[i])
        cluster_embeddings.append(model.encode(cluster_text))

    top_words_embedding = model.encode([top_words])
    similarity_matrix = cosine_similarity(top_words_embedding, cluster_embeddings)
    highest_scored_cluster = np.argmax(similarity_matrix, axis=1)[0]
    highest_scored_clusters.append(highest_scored_cluster)
    top_matched_cluster_keywords.append(', '.join(word_clusters[highest_scored_cluster]))

df['highest_scored_cluster'] = highest_scored_clusters
df['top_matched_cluster_keywords'] = top_matched_cluster_keywords

C:\Users\imran\.conda\envs\myenv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\imran\.conda\envs\myenv\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=1.
  warnings.warn(
C:\Users\imran\.conda\envs\myenv\lib\site-packages\sklearn\cluster\_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
C:\Users\imran\.conda\envs\myenv\lib\site-packages\sklearn\cluster\_kmeans.py:1382: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can a

In [68]:
df

,name,cb_description,highest_scored_cluster,top_matched_cluster_keywords
0,InterResolve,interresolve radically new approach deal car a...,3,"insurer, agency, insurance"
1,GladCloud,gladcloud trade marketing infrastructure combi...,0,"solution, product, cosmetic, location, customi..."
2,13th-Lab,lab develop generation computer vision platfor...,5,"computer, vehicle, store, owner, originally, d..."
3,1939-Games,games indie game development studio base reykj...,4,"gaming, game, fun, games, experience, fuse"
4,2021.AI,2021.ai serve grow business need oversight man...,5,"complex, model, expertise, ai, science, data"
5,20nine,20nine helped transform regional national glob...,5,"millwork, athletic, industry, category, market..."
6,21GRAMS,gram offer postal management corporate custome...,0,"postage, mailing, post, communication, postal"
7,Geltor,geltor conscious biodesign company create worl...,6,"company, protein, fermentation, gmo, cultivate"
8,21st.BIO,bio bioproduction startup assist bioindustrial...,5,"bioindustrial, bioproduction, bio, molecule"
9,iSIZE-Technologies,isize technology company deliver deep neural n...,9,"isize, bitrate, encoder"


In [69]:
df.to_csv(r"C:\Users\imran\DataspellProjects\WalidCase\data\processed\30startups_with_clusters_and_desc_kws_v2.csv")

In [ ]:
# Cluster the unique words using KMeans
n_clusters = 10
kmeans = KMeans(n_clusters=n_clusters, random_state=42)
kmeans_output = kmeans.fit_predict(word_embeddings)

# Group the unique words by their assigned clusters
word_clusters = {i: [] for i in range(n_clusters)}
for i, label in enumerate(kmeans_output):
    word_clusters[label].append(unique_words[i])

In [ ]:
# Step 4: Use a sentence transformer to make embeddings for each cluster
cluster_embeddings = []
for i in range(n_clusters):
    cluster_text = ', '.join(word_clusters[i])
    cluster_embeddings.append(model.encode(cluster_text))

In [ ]:
# Step 5: Find the cosine similarity between each cluster and the whole description embeddings
similarity_matrix = cosine_similarity(description_embeddings, cluster_embeddings)
df['highest_scored_cluster'] = np.argmax(similarity_matrix, axis=1)

In [ ]:
# Append the top matched cluster keywords to the corresponding row
df['top_matched_cluster_keywords'] = df['highest_scored_cluster'].apply(lambda x: ', '.join(word_clusters[x]))


In [28]:
df

,id,name,cb_description,highest_scored_cluster,top_matched_cluster_keywords
0,4081,InterResolve,interresolve radically new approach deal car a...,3,"convert, convenient, open, provision, planar, ..."
1,2785,GladCloud,gladcloud trade marketing infrastructure combi...,0,"valuation, offering, financial, marketplace, a..."
2,23680,13th-Lab,lab develop generation computer vision platfor...,3,"convert, convenient, open, provision, planar, ..."
3,2932,Hilson-Moran,hilson moran provide consultancy building serv...,3,"convert, convenient, open, provision, planar, ..."
4,22003,1928-Diagnostics,diagnostic digital health company crunch dna d...,4,"chemnitz, pharmaceuticals, clinical, deinestud..."
...,...,...,...,...,...
95,23887,Acerta-Analytics,acerta machine learn artificial intelligence p...,5,"competitor, capability, momentum, speed, condi..."
96,3202,Cúram-Software,cúram software develop offer social enterprise...,3,"convert, convenient, open, provision, planar, ..."
97,25054,Acknoledger,ackno api consist ucns ackno plagiarism engine...,3,"convert, convenient, open, provision, planar, ..."
98,6721,Adviva,adviva operate european online advertising net...,5,"competitor, capability, momentum, speed, condi..."


,id,name,cb_description,keyword_cluster,similar_cluster
0,4081,InterResolve,interresolve radically new approach deal car a...,1,1
1,2785,GladCloud,gladcloud trade marketing infrastructure combi...,1,1
2,23680,13th-Lab,lab develop generation computer vision platfor...,0,0
3,2932,Hilson-Moran,hilson moran provide consultancy building serv...,1,1
4,22003,1928-Diagnostics,diagnostic digital health company crunch dna d...,0,0
...,...,...,...,...,...
95,23887,Acerta-Analytics,acerta machine learn artificial intelligence p...,0,0
96,3202,Cúram-Software,cúram software develop offer social enterprise...,1,0
97,25054,Acknoledger,ackno api consist ucns ackno plagiarism engine...,1,0
98,6721,Adviva,adviva operate european online advertising net...,1,1
